In [11]:
import os
import numpy as np

In [12]:
graph_signal_matrix_filename = '/home/yannan1/or_projects/traffic_flow_prediciton/data/PEMS04.npz'
data = np.load(graph_signal_matrix_filename)
data['data'].shape

(16992, 307, 3)

它的形状是# (sequence_length, num_of_vertices, num_of_features)（16992, 307, 3）

我们有307个检测器，每个检测器检测3个特征，如车速、拥堵等，每5分钟存储一次数据。5分钟后，我们的数据维度是（1,307,3），1小时后的数据维度是（12,307,3），1天后的数据维度是（12 * 24,307,3），59天后的数据维度是(12 * 24 * 59,307,3)= （16992,307,3）

1. **原始数据**：数据来自 Kaggle 数据集，形状为(16992, 307, 3) ，由 307 个探测器每 5 分钟采集车速、拥堵等 3 类特征，经 59 天积累得到。
2. **采样策略**：为预测未来交通，设计三种采样构建模型，结果融合作最终输出，具体为：
    - **近期片段**：用过去两小时（如 6:00 - 8:00 ）数据，基于交通拥堵渐进性，预测当下（如 8:00 ）交通。
    - **每日周期片段**：取上周、昨天、前天同一小时（如周二、周三 8:00 ）数据，依据日常规律下的交通重复模式（如早高峰 ），预测当日同小时（如周四 8:00 ）交通。
    - **每周周期片段**：用上上周、上周同一小时（如过去两周周四 8:00 ）数据，因周一等工作日交通模式与历史同期相似、异于周末，以此捕捉周周期性，预测本周同小时（如本周四 8:00 ）交通。
3. **数据处理与示例**：示例按小时采样，每小时（12 个 5 分钟点，形状 (12, 307, 3) ）数据预测下一小时数据，通过 `get_sample_indices` 等函数从原始 16992 个点中提取输入（如 `original[0:12]` ）和目标（如 `original[12:24]` ）数据，后移 5 分钟循环采样，因首尾部分数据无法用于预测或被预测，最终得到 16969 组有效示例（16992 - 23 ）。 
4. **数据划分**：10181 组用于训练集（35 天 ）、3394 组用于验证集（12 天 ）、3394 组用于测试集（12 天 ） ，以此支撑模型训练、调优与评估。 


In [13]:
def search_data(sequence_length, num_of_depend, label_start_idx,num_for_predict, units, points_per_hour):
    '''
    Parameters
    ----------
    sequence_length: int, length of all history data
    num_of_depend: int,
    label_start_idx: int, the first index of predicting target
    num_for_predict: int, the number of points will be predicted for each sample
    units: int, week: 7 * 24, day: 24, recent(hour): 1
    points_per_hour: int, number of points per hour, depends on data
    Returns
    ----------
    list[(start_idx, end_idx)]
    '''

    if points_per_hour < 0:
        raise ValueError("points_per_hour should be greater than 0!")

    if label_start_idx + num_for_predict > sequence_length:
        return None

    x_idx = []
    for i in range(1, num_of_depend + 1):
        start_idx = label_start_idx - points_per_hour * units * i
        end_idx = start_idx + num_for_predict
        if start_idx >= 0:
            x_idx.append((start_idx, end_idx))
        else:
            return None

    if len(x_idx) != num_of_depend:
        return None

    return x_idx[::-1]
def get_sample_indices(data_sequence, num_of_weeks, num_of_days, num_of_hours, label_start_idx, num_for_predict, points_per_hour=12):
    '''
    Parameters
    ----------
    data_sequence: np.ndarray shape is (sequence_length, num_of_vertices, num_of_features)
    num_of_weeks, num_of_days, num_of_hours: int
    label_start_idx: int, the first index of predicting target
    num_for_predict: int,the number of points will be predicted for each sample
    points_per_hour: int, default 12, number of points per hour
    Returns
    ----------
    week_sample: np.ndarray shape is (num_of_weeks * points_per_hour, num_of_vertices, num_of_features)
    day_sample: np.ndarray shape is (num_of_days * points_per_hour,  num_of_vertices, num_of_features)
    hour_sample: np.ndarray   shape is (num_of_hours * points_per_hour, num_of_vertices, num_of_features)
    target: np.ndarray shape is (num_for_predict, num_of_vertices, num_of_features)
    '''
    week_sample, day_sample, hour_sample = None, None, None
#------------------------------------Ignore
    if label_start_idx + num_for_predict > data_sequence.shape[0]: 
        return week_sample, day_sample, hour_sample, None

    if num_of_weeks > 0:
        week_indices = search_data(data_sequence.shape[0], num_of_weeks, label_start_idx, num_for_predict,7 * 24, points_per_hour)
        if not week_indices:
            return None, None, None, None

        week_sample = np.concatenate([data_sequence[i: j] for i, j in week_indices], axis=0)

    if num_of_days > 0:
        day_indices = search_data(data_sequence.shape[0], num_of_days,  label_start_idx, num_for_predict, 24, points_per_hour)
        if not day_indices:
            return None, None, None, None

        day_sample = np.concatenate([data_sequence[i: j] for i, j in day_indices], axis=0)
#----------------------------------Continue
    if num_of_hours > 0:
        hour_indices = search_data(data_sequence.shape[0], num_of_hours, label_start_idx, num_for_predict, 1, points_per_hour)
        if not hour_indices:
            return None, None, None, None
        hour_sample = np.concatenate([data_sequence[i: j] for i, j in hour_indices], axis=0)
    
    if num_of_hours > 10:
        return 1;
    target = data_sequence[label_start_idx: label_start_idx + num_for_predict]

    return week_sample, day_sample, hour_sample, target
def read_and_generate_dataset(graph_signal_matrix_filename, num_of_weeks, num_of_days, num_of_hours, num_for_predict, points_per_hour=12):
    '''
    Parameters
    ----------
    graph_signal_matrix_filename: str, path of graph signal matrix file
    num_of_weeks, num_of_days, num_of_hours: int
    num_for_predict: int
    points_per_hour: int, default 12, depends on data
    Returns
    ----------
    feature: np.ndarray, shape is (num_of_samples, num_of_depend * points_per_hour, num_of_vertices, num_of_features)
    target: np.ndarray, shape is (num_of_samples, num_of_vertices, num_for_predict)
    '''
    #--------------------------------- Read original data 
    data_seq = np.load(graph_signal_matrix_filename)['data']  # (sequence_length, num_of_vertices, num_of_features) (16992, 307, 3)
    
    #---------------------------------
    all_samples = []
    for idx in range(data_seq.shape[0]):
        sample = get_sample_indices(data_seq, num_of_weeks, num_of_days, num_of_hours, idx, num_for_predict, points_per_hour)
        if ((sample[0] is None) and (sample[1] is None) and (sample[2] is None)):
            continue

        week_sample, day_sample, hour_sample, target = sample #  week_sample, day_sample are None because we are predicting per hour
        #print(target.shape) # hour_sample and target (12, 307, 3)
        sample = []  # [(week_sample),(day_sample),(hour_sample),target,time_sample]
#-------------------------------- Ignore
        if num_of_weeks > 0:
            week_sample = np.expand_dims(week_sample, axis=0).transpose((0, 2, 3, 1))  # 变成维度为 (1,num_of_vertices,num_of_features,sequence_length)
            sample.append(week_sample)

        if num_of_days > 0:
            day_sample = np.expand_dims(day_sample, axis=0).transpose((0, 2, 3, 1))  # 变成维度为 (1,num_of_vertices,num_of_features,sequence_length)
            sample.append(day_sample)
#----------------------------------Continue
        if num_of_hours > 0:
            hour_sample = np.expand_dims(hour_sample, axis=0).transpose((0, 2, 3, 1))  # 变成维度为 (1,num_of_vertices,num_of_features,sequence_length)
            sample.append(hour_sample)

        target = np.expand_dims(target, axis=0).transpose((0, 2, 3, 1))[:, :, 0, :]  # 通过 [:, :, 0, :] 提取该时间步的目标值
        sample.append(target)
        time_sample = np.expand_dims(np.array([idx]), axis=0)  # (1,1) # 把时间索引也加入
        sample.append(time_sample)
        all_samples.append(sample)#sampe：[(week_sample),(day_sample),(hour_sample),target,time_sample] = [(1,N,F,Tw),(1,N,F,Td),(1,N,F,Th),(1,N,Tpre),(1,1)]

    split_line1 = int(len(all_samples) * 0.6)
    split_line2 = int(len(all_samples) * 0.8)

    training_set = [np.concatenate(i, axis=0)  for i in zip(*all_samples[:split_line1])] #[(B,N,F,Tw),(B,N,F,Td),(B,N,F,Th),(B,N,Tpre),(B,1)]
    validation_set = [np.concatenate(i, axis=0) for i in zip(*all_samples[split_line1: split_line2])]
    testing_set = [np.concatenate(i, axis=0) for i in zip(*all_samples[split_line2:])]

    return training_set, validation_set, testing_set

### 数据处理与数据集生成流程解析

#### 1. 数据读取与基本信息

首先从文件中读取原始数据：
```python
data_seq = np.load(graph_signal_matrix_filename)['data']  # 形状 (16992, 307, 3)
```
- `16992`：时间步总数（59天，每天24小时，每小时12个时间点：59×24×12=16992）
- `307`：探测器数量
- `3`：每个探测器的特征数（如车速、流量、占有率）


#### 2. 样本生成核心函数：get_sample_indices

该函数为每个时间点生成三类样本（周周期、日周期、近期）和目标值：

```python
def get_sample_indices(...):
    # 生成周周期样本
    if num_of_weeks > 0:
        week_indices = search_data(..., units=7*24, ...)  # units=7*24表示每周
        week_sample = np.concatenate([data_sequence[i:j] for i,j in week_indices], axis=0)
    
    # 生成日周期样本
    if num_of_days > 0:
        day_indices = search_data(..., units=24, ...)  # units=24表示每日
        day_sample = np.concatenate([data_sequence[i:j] for i,j in day_indices], axis=0)
    
    # 生成近期样本
    if num_of_hours > 0:
        hour_indices = search_data(..., units=1, ...)  # units=1表示每小时
        hour_sample = np.concatenate([data_sequence[i:j] for i,j in hour_indices], axis=0)
    
    # 获取目标值
    target = data_sequence[label_start_idx: label_start_idx + num_for_predict]
    return week_sample, day_sample, hour_sample, target
```

#### 3. search_data函数：确定时间窗口

`search_data`函数根据时间周期（周/日/小时）计算历史数据窗口：
- **输入**：当前预测起点`label_start_idx`，周期单位`units`（如7×24表示每周）
- **输出**：历史时间窗口列表`[(start_idx, end_idx), ...]`

**示例**：若`num_of_hours=2`（获取2小时历史数据），`points_per_hour=12`（每小时12个点），当前预测起点是`idx=24`：
- 第一个窗口：`24 - 12×1×1 = 12` 到 `12 + num_for_predict`
- 第二个窗口：`24 - 12×1×2 = 0` 到 `0 + num_for_predict`
- 窗口列表：`[(12, 12+num_for_predict), (0, 0+num_for_predict)]`


#### 4. 样本处理与存储：all_samples

通过循环每个时间点`idx`生成样本：
```python
all_samples = []
for idx in range(data_seq.shape[0]):
    sample = get_sample_indices(...)
    if sample有效:
        week_sample, day_sample, hour_sample, target = sample
        
        # 处理各周期样本维度
        if num_of_weeks > 0:
            week_sample = np.expand_dims(week_sample, axis=0).transpose((0, 2, 3, 1))
        if num_of_days > 0:
            day_sample = np.expand_dims(day_sample, axis=0).transpose((0, 2, 3, 1))
        if num_of_hours > 0:
            hour_sample = np.expand_dims(hour_sample, axis=0).transpose((0, 2, 3, 1))
        
        # 处理目标值
        target = np.expand_dims(target, axis=0).transpose((0, 2, 3, 1))[:, :, 0, :]
        
        # 添加时间索引
        time_sample = np.expand_dims(np.array([idx]), axis=0)  # (1,1)
        
        # 保存样本
        all_samples.append([week_sample, day_sample, hour_sample, target, time_sample])
```

#### 5. 维度变换详解

以`hour_sample`为例，假设原始形状为`(24, 307, 3)`（2小时数据，每小时12个点）：
1. `np.expand_dims(hour_sample, axis=0)` → 形状变为`(1, 24, 307, 3)`
2. `transpose((0, 2, 3, 1))` → 维度重排为`(1, 307, 3, 24)`
   - 含义：`[批次, 探测器, 特征, 时间步]`

目标值`target`处理：
- 原始形状`(12, 307, 3)`（预测1小时，12个时间点）
- 处理后形状`(1, 307, 12)`（通过`[:, :, 0, :]`提取特征维度为1）


#### 6. 数据集划分：training_set, validation_set, testing_set

```python
split_line1 = int(len(all_samples) * 0.6)     # 60%训练集
split_line2 = int(len(all_samples) * 0.8)     # 80%处划分验证集和测试集

training_set = [np.concatenate(i, axis=0) for i in zip(*all_samples[:split_line1])]
validation_set = [np.concatenate(i, axis=0) for i in zip(*all_samples[split_line1: split_line2])]
testing_set = [np.concatenate(i, axis=0) for i in zip(*all_samples[split_line2:])]
```

#### 7. 数据集生成过程解析

1. **zip(*all_samples[:split_line1])**：
   - 将`all_samples`前60%的样本按特征位置重组
   - 例如，所有样本的`week_sample`组成一组，`day_sample`组成另一组

2. **np.concatenate(i, axis=0)**：
   - 沿第0轴（批次维度）拼接
   - 假设每个样本的`hour_sample`形状为`(1, 307, 3, 24)`，拼接后形状为`(B, 307, 3, 24)`，其中B是训练集样本数

3. **最终数据集结构**：
   - `training_set`、`validation_set`、`testing_set`结构相同，均为包含5个元素的列表：
     1. 周周期样本：`(B, 307, 3, Tw)`
     2. 日周期样本：`(B, 307, 3, Td)`
     3. 近期样本：`(B, 307, 3, Th)`
     4. 目标值：`(B, 307, Tpre)`
     5. 时间索引：`(B, 1)`


### 完整维度变换示例

假设：
- `num_of_weeks=1`，`num_of_days=1`，`num_of_hours=2`
- `num_for_predict=12`（预测1小时）
- 训练集样本数`B=10181`

#### 1. 单样本处理后维度：
- `week_sample`：`(1, 307, 3, 7×24×12)` → 实际可能根据`num_of_weeks`调整
- `day_sample`：`(1, 307, 3, 24×12)`
- `hour_sample`：`(1, 307, 3, 2×12)`
- `target`：`(1, 307, 12)`
- `time_sample`：`(1, 1)`

#### 2. 训练集拼接后维度：
- `training_set[0]`（周周期）：`(10181, 307, 3, 7×24×12)`
- `training_set[1]`（日周期）：`(10181, 307, 3, 24×12)`
- `training_set[2]`（近期）：`(10181, 307, 3, 2×12)`
- `training_set[3]`（目标值）：`(10181, 307, 12)`
- `training_set[4]`（时间索引）：`(10181, 1)`

这种处理方式能够有效捕捉交通数据的时间周期性（周/日/小时），为后续的时空模型提供多维度特征输入。

In [ ]:
num_of_vertices = 307
points_per_hour = 12
num_for_predict = 12
num_of_weeks = 0
num_of_days = 0
num_of_hours = 1

data_seq = np.load(graph_signal_matrix_filename)['data']  # (sequence_length, num_of_vertices, num_of_features) (16992, 307, 3)
    
training_set, validation_set, testing_set = read_and_generate_dataset(graph_signal_matrix_filename, 0, 0, num_of_hours, 
                                                                      num_for_predict, points_per_hour=points_per_hour)

In [32]:
def normalization(train, val, test):
    '''
    Parameters
    ----------
    train, val, test: np.ndarray (B,N,F,T)
    Returns
    ----------
    stats: dict, two keys: mean and std
    train_norm, val_norm, test_norm: np.ndarray,
                                     shape is the same as original
    '''

    assert train.shape[1:] == val.shape[1:] and val.shape[1:] == test.shape[1:]  # ensure the num of nodes is the same
    mean = train.mean(axis=(0,1,3), keepdims=True)
    std = train.std(axis=(0,1,3), keepdims=True)
    print('mean.shape:',mean.shape)
    print('std.shape:',std.shape)

    def normalize(x):
        return (x - mean) / std

    train_norm = normalize(train)
    val_norm = normalize(val)
    test_norm = normalize(test)

    return {'_mean': mean, '_std': std}, train_norm, val_norm, test_norm
train_x = np.concatenate(training_set[:-2], axis=-1)  # (B,N,F,T')
val_x = np.concatenate(validation_set[:-2], axis=-1)
test_x = np.concatenate(testing_set[:-2], axis=-1)

train_target = training_set[-2]  # (B,N,T)
val_target = validation_set[-2]
test_target = testing_set[-2]

train_timestamp = training_set[-1]  # (B,1)
val_timestamp = validation_set[-1]
test_timestamp = testing_set[-1]

(stats, train_x_norm, val_x_norm, test_x_norm) = normalization(train_x, val_x, test_x)

all_data = {'train': { 'x': train_x_norm, 'target': train_target,'timestamp': train_timestamp},
            'val': {'x': val_x_norm, 'target': val_target, 'timestamp': val_timestamp},
            'test': {'x': test_x_norm, 'target': test_target, 'timestamp': test_timestamp},
            'stats': {'_mean': stats['_mean'], '_std': stats['_std']} }

print('train x:', all_data['train']['x'].shape)
print('train target:', all_data['train']['target'].shape)
print('train timestamp:', all_data['train']['timestamp'].shape)
print()
print('val x:', all_data['val']['x'].shape)
print('val target:', all_data['val']['target'].shape)
print('val timestamp:', all_data['val']['timestamp'].shape)
print()
print('test x:', all_data['test']['x'].shape)
print('test target:', all_data['test']['target'].shape)
print('test timestamp:', all_data['test']['timestamp'].shape)
print()
print('train data _mean :', all_data['stats']['_mean'].shape, all_data['stats']['_mean'])
print('train data _std :', all_data['stats']['_std'].shape, all_data['stats']['_std'])

mean.shape: (1, 1, 3, 1)
std.shape: (1, 1, 3, 1)
train x: (10181, 307, 3, 12)
train target: (10181, 307, 12)
train timestamp: (10181, 1)

val x: (3394, 307, 3, 12)
val target: (3394, 307, 12)
val timestamp: (3394, 1)

test x: (3394, 307, 3, 12)
test target: (3394, 307, 12)
test timestamp: (3394, 1)

train data _mean : (1, 1, 3, 1) [[[[2.07227338e+02]
   [5.13195612e-02]
   [6.34740574e+01]]]]
train data _std : (1, 1, 3, 1) [[[[1.56477655e+02]
   [4.78541626e-02]
   [8.10351724e+00]]]]


In [33]:
file = os.path.basename(graph_signal_matrix_filename).split('.')[0]
dirpath = './data/'
filename = os.path.join(dirpath, file + '_r' + str(num_of_hours) + '_d' + str(num_of_days) + '_w' + str(num_of_weeks)) + '_astcgn'
print('save file:', filename)
np.savez_compressed(filename,
                train_x=all_data['train']['x'],train_target=all_data['train']['target'],train_timestamp=all_data['train']['timestamp'],
                val_x=all_data['val']['x'], val_target=all_data['val']['target'],val_timestamp=all_data['val']['timestamp'],
                test_x=all_data['test']['x'], test_target=all_data['test']['target'], test_timestamp=all_data['test']['timestamp'],
                mean=all_data['stats']['_mean'], std=all_data['stats']['_std'])

save file: ./PEMS04_r1_d0_w0_astcgn
